# Doing cool stuff with satellite data!
What am I doing?

## Checklist of steps

- [ ] Access Google Earth Engine API
- [ ] Download LandSat for growing season in Snake River basin

## Resources for datasets and usage
Google: Earth Engine Developers docs (http://developers.google.com/earth-engine)

NASA: Landsat 8 Bands (https://landsat.gsfc.nasa.gov/landsat-8/landsat-8-bands)

USGS: What are the best Landsat spectral bands for use in my research? (https://www.usgs.gov/faqs/what-are-best-landsat-spectral-bands-use-my-research?qt-news_science_products=0#qt-news_science_products)

USGS: Common Landsat Band RGB Composites (https://www.usgs.gov/media/images/common-landsat-band-rgb-composites)

NASA: How to Interpret Common False Color Images (https://earthobservatory.nasa.gov/features/FalseColor/page6.php)

USGS: Spectral Characteristics Viewer (https://landsat.usgs.gov/spectral-characteristics-viewer)

What we built in the class: Karin’s “Landsat 8 Analyzer” code snippet (https://code.earthengine.google.com/ad2cf9683d62290dcbe8d3b00babe45f)

## Setup

### Install Google Earth Engine API

Must start by following the instructions here: https://developers.google.com/earth-engine/guides/python_install-conda#mac_5

Once that is all good to go you should make sure your `ee` conda environment is activated (you should see `(ee)` at the beginning of your command prompt in the terminal where your username usually is. Get there by running `conda activate ee` (after creating the conda env according to the instructions above).

Make sure you are always in your `ee` conda environment for development, and install all library dependencies only after activating your environment.

To share the environment with other users for reproducibility, run 
`conda env export > $HOME/ee-shared-env.yml`, then have the other person download that file and run `conda env create -f path-to-ee-shared-env.yml`

Once that is all set up and you are in your conda env you can simply import earthengine (`ee`)

### Import and test GEE

In [2]:
import ee

# Initialize the Earth Engine module.
ee.Initialize()

# Print metadata for a DEM dataset.
print(ee.Image('USGS/SRTMGL1_003').getInfo())

{'type': 'Image', 'bands': [{'id': 'elevation', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [1296001, 417601], 'crs': 'EPSG:4326', 'crs_transform': [0.0002777777777777778, 0, -180.0001388888889, 0, -0.0002777777777777778, 60.00013888888889]}], 'id': 'USGS/SRTMGL1_003', 'version': 1605362602494378, 'properties': {'system:visualization_0_min': '0.0', 'type_name': 'Image', 'thumb': 'https://mw1.google.com/ges/dd/images/SRTM90_V4_thumb.png', 'description': '<p>The Shuttle Radar Topography Mission (SRTM, see <a href="http://onlinelibrary.wiley.com/doi/10.1029/2005RG000183/full">Farr\net al. 2007</a>)\ndigital elevation data is an international research effort that\nobtained digital elevation models on a near-global scale. This\nSRTM V3 product (SRTM Plus) is provided by NASA JPL\nat a resolution of 1 arc-second (approximately 30m).</p><p>This dataset has undergone a void-filling process using open-source data\n(ASTER GDEM2, GMTED2010, a

### Success!!!

## Testing an example use case
## NOTE: Not sure if this is useful AT ALL
Source: https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/Earth_Engine_asset_from_cloud_geotiff.ipynb

In [6]:
from ee import oauth
from google_auth_oauthlib.flow import Flow
import json

# Build the `client_secrets.json` file by borrowing the
# Earth Engine python authenticator.
client_secrets = {
    'web': {
        'client_id': oauth.CLIENT_ID,
        'client_secret': oauth.CLIENT_SECRET,
        'redirect_uris': [oauth.REDIRECT_URI],
        'auth_uri': 'https://accounts.google.com/o/oauth2/auth',
        'token_uri': 'https://accounts.google.com/o/oauth2/token'
    }
}

# Write to a json file.
client_secrets_file = 'client_secrets.json'
with open(client_secrets_file, 'w') as f:
  json.dump(client_secrets, f, indent=2)

# Start the flow using the client_secrets.json file.
flow = Flow.from_client_secrets_file(client_secrets_file,
    scopes=oauth.SCOPES,
    redirect_uri=oauth.REDIRECT_URI)

# Get the authorization URL from the flow.
auth_url, _ = flow.authorization_url(prompt='consent')

# Print instructions to go to the authorization URL.
oauth._display_auth_instructions_with_print(auth_url)
print('\n')

# The user will get an authorization code.
# This code is used to get the access token.
code = input('Enter the authorization code: \n')
flow.fetch_token(code=code)

# Get an authorized session from the flow.
session = flow.authorized_session()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&state=WAc7fOl6xgAtrPdIGGHMOrNl1lwGY1&prompt=consent&access_type=offline

The authorization workflow will generate a code, which you should paste in the box below. 


Enter the authorization code: 
4/1AY0e-g4r65fEw4WphgsJtkbnDzn0i9G9tHMhD0AO7KUoB_S13KB_6GU-8V4


## Generating datasets/assets in GEE then exporting them for use here
Useful info: https://developers.google.com/earth-engine/guides/exporting#configuration-parameters

My understanding is that the best way to do this is to use the JavaScript API in GEE to access and process datasets, which can then be exported and loaded here for use in models, etc.

Still figuring this out though.

### Loading an existing COG (Cloud Optimized GeoTiff)
I thiiiiiink then you can access the asset using something like the following (which uses an example COG, or Cloud Optimized GeoTiff). The request body is an instance of an EarthEngineAsset.

This example COG was generated using this example script: https://code.earthengine.google.com/?scriptPath=Examples%3ACloud%20Masking%2FSentinel2

In [7]:
# Request body as a dictionary.
request = {
  'type': 'IMAGE',
  'gcs_location': {
    'uris': ['gs://ee-docs-demos/COG_demo.tif']
  },
  'properties': {
    'source': 'https://code.earthengine.google.com/d541cf8b268b2f9d8f834c255698201d'
  },
  'startTime': '2016-01-01T00:00:00.000000000Z',
  'endTime': '2016-12-31T15:01:23.000000000Z',
}

from pprint import pprint
pprint(json.dumps(request))

('{"type": "IMAGE", "gcs_location": {"uris": '
 '["gs://ee-docs-demos/COG_demo.tif"]}, "properties": {"source": '
 '"https://code.earthengine.google.com/d541cf8b268b2f9d8f834c255698201d"}, '
 '"startTime": "2016-01-01T00:00:00.000000000Z", "endTime": '
 '"2016-12-31T15:01:23.000000000Z"}')


### Send the request
Make the POST request to the Earth Engine CreateAsset endpoint.

This will place the asset in your assets folder of your GEE account.

In [9]:
# Send the request
# Where Earth Engine assets are kept.
project_folder = 'earthengine-legacy'
# Your user folder name and new asset name.
asset_id = 'users/elliothohn/sample_asset'

url = 'https://earthengine.googleapis.com/v1alpha/projects/{}/assets?assetId={}'

response = session.post(
  url = url.format(project_folder, asset_id),
  data = json.dumps(request)
)

pprint(json.loads(response.content))

{'bands': [{'dataType': {'precision': 'INT', 'range': {'max': 255}},
            'grid': {'affineTransform': {'scaleX': 8.983152841195215e-05,
                                         'scaleY': -8.983152841195215e-05,
                                         'translateX': 6.236014870829306,
                                         'translateY': 46.32773616955514},
                     'crsCode': 'EPSG:4326',
                     'dimensions': {'height': 445, 'width': 1377}},
            'id': 'vis-red',
            'pyramidingPolicy': 'MEAN'},
           {'dataType': {'precision': 'INT', 'range': {'max': 255}},
            'grid': {'affineTransform': {'scaleX': 8.983152841195215e-05,
                                         'scaleY': -8.983152841195215e-05,
                                         'translateX': 6.236014870829306,
                                         'translateY': 46.32773616955514},
                     'crsCode': 'EPSG:4326',
                     'dimensions': {'h

## Example 2 - uploading a set of image tiles into a single asset using a manifest file

Source: https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/Uploading_image_tiles_as_a_single_asset_using_a_manifest.ipynb

#### NOTE: this is not working right now, and I don't think I actually need it quite yet (because I think I can start by using assets within GEE without having to store anything in Google Cloud Storage yet). Once I'm at that stage (assuming that's actually necessary/useful?) maybe move this into it's own notebook.

In [12]:
# URLs of a few tiles.
urls = [
  'http://data.ess.tsinghua.edu.cn/data/fromglc10_2017v01/fromglc10v01_36_-120.tif',
  'http://data.ess.tsinghua.edu.cn/data/fromglc10_2017v01/fromglc10v01_36_-122.tif',
  'http://data.ess.tsinghua.edu.cn/data/fromglc10_2017v01/fromglc10v01_36_-124.tif',
  'http://data.ess.tsinghua.edu.cn/data/fromglc10_2017v01/fromglc10v01_38_-120.tif',
  'http://data.ess.tsinghua.edu.cn/data/fromglc10_2017v01/fromglc10v01_38_-122.tif',
  'http://data.ess.tsinghua.edu.cn/data/fromglc10_2017v01/fromglc10v01_38_-124.tif'
]

# You need to have write access to this bucket.
bucket = 'imagery-testing'

# Pipe curl output to gsutil.
for f in urls:
  filepath = bucket + '/' + f.split('/')[-1]
  !curl {f} | gsutil cp - {filepath}

/bin/bash: gsutil: command not found
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0 58.1M    0 22919    0     0  42208      0  0:24:03 --:--:--  0:24:03 42208
curl: (23) Failed writing body (0 != 11004)
/bin/bash: gsutil: command not found
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0 48.0M    0 36251    0     0  52310      0  0:16:03 --:--:--  0:16:03 52385
curl: (23) Failed writing body (0 != 16384)
/bin/bash: gsutil: command not found
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0 11.3M    0 36252    0     0  48530      0  0:04:04 --:--:--  0:04:04 48530
curl: (23) Failed writing body (0 != 16384)
/bin/bash: gsutil: command not found
  % Total

## TensorFlow example
Source: https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/Earth_Engine_TensorFlow_AI_Platform.ipynb


In [13]:
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AY0e-g7kyUwxxjzbUe1Uol-n3PkjG_sF6uNIDujJfXBIRomrH85ZEuk3TIg

Successfully saved authorization token.


In [17]:
import tensorflow as tf
print(tf.__version__)

ModuleNotFoundError: No module named 'tensorflow'